<a href="https://colab.research.google.com/github/bubibon/AI_ART/blob/master/FID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FID evaluation

FID computation implementation by 

https://github.com/mseitzer/pytorch-fid

In [ ]:
!pip install pytorch-fid

  Created wheel for pytorch-fid: filename=pytorch_fid-0.2.1-py3-none-any.whl size=14835 sha256=3cef7076634bc2959ea3eb3350589967922621e563213517958c9466e7e54633
  Stored in directory: /root/.cache/pip/wheels/24/ac/03/c5634775c8a64f702343ef5923278f8d3bb8c651debc4a6890
Successfully built pytorch-fid


In [ ]:
!python -m pytorch_fid --help

usage: __main__.py [-h] [--batch-size BATCH_SIZE] [--num-workers NUM_WORKERS]
                   [--device DEVICE] [--dims {64,192,768,2048}]
                   path path

positional arguments:
  path                  Paths to the generated images or to .npz statistic
                        files

optional arguments:
  -h, --help            show this help message and exit
  --batch-size BATCH_SIZE
                        Batch size to use (default: 50)
  --num-workers NUM_WORKERS
                        Number of processes to use for data loading. Defaults
                        to `min(8, num_cpus)` (default: None)
  --device DEVICE       Device to use. Like cuda, cuda:0 or cpu (default:
                        None)
  --dims {64,192,768,2048}
                        Dimensionality of Inception features to use. By
                        default, uses pool3 features (default: 2048)


Here we are trying first to compute FID for the whole dataset vs 5000 generates sample images from the various models. 

In [ ]:
!python -m pytorch_fid /content/drive/MyDrive/impressionist_landscapes_resized_1024 /content/drive/MyDrive/GENERATED_IMAGES/FROM_SCRATCH --device cuda:0

100% 100/100 [03:05<00:00,  1.85s/it]
100% 100/100 [09:33<00:00,  5.74s/it]
FID:  31.551734299874624


In [ ]:
!python -m pytorch_fid /content/drive/MyDrive/impressionist_landscapes_resized_1024 /content/drive/MyDrive/GENERATED_IMAGES/TRANSFER_L --device cuda:0

100% 100/100 [01:40<00:00,  1.00s/it]
100% 100/100 [13:13<00:00,  7.93s/it]
FID:  25.957871990116473


In [ ]:
!python -m pytorch_fid /content/drive/MyDrive/impressionist_landscapes_resized_1024 /content/drive/MyDrive/GENERATED_IMAGES/TF_FREEZED --device cuda:0

100% 100/100 [01:40<00:00,  1.00s/it]
100% 100/100 [10:32<00:00,  6.32s/it]
FID:  69.35459961831333


For a more realistic FID computation, the dataset of real images should not include the 4500 ones used for training the models. For this reason it is more appropriate to compare 500 generated samples against the previously extracted test set of 500 real paintings. The problem in this case is related to its computation, 500 i

In [ ]:
#Creating a temporary directory containing 500 random generated samples
%mkdir /content/scratch
dirpath= '/content/drive/MyDrive/GENERATED_IMAGES/FROM_SCRATCH/'
filenames = random.sample(os.listdir(dirpath), 500) 
for fname in filenames:
  source = dirpath+fname
  destination = '/content/scratch/' + fname
  shutil.copy(source, destination)

len(os.listdir('/content/scratch/'))


In [ ]:
!python -m pytorch_fid /content/drive/MyDrive/Data/test /content/scratch/ --device cuda:0

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 132MB/s]
100% 10/10 [00:13<00:00,  1.31s/it]
100% 10/10 [00:16<00:00,  1.64s/it]
FID:  75.72903789654325


In [ ]:
#Creating a temporary directory containing 500 random generated samples
%mkdir /content/tf
dirpath= '/content/drive/MyDrive/GENERATED_IMAGES/TRANSFER_L/'
filenames = random.sample(os.listdir(dirpath), 500) 
for fname in filenames:
  source = dirpath+fname
  destination = '/content/tf/' + fname
  shutil.copy(source, destination)

len(os.listdir('/content/tf/'))

In [ ]:
!python -m pytorch_fid /content/drive/MyDrive/Data/test /content/tf/ --device cuda:0

100% 10/10 [00:12<00:00,  1.21s/it]
100% 10/10 [00:16<00:00,  1.67s/it]
FID:  73.10920139740375


The calculated FID seem very high. This is probably due to the fact that 500 is an incredibly small sample dataset size for FID computation. To give a proof of that, we are going to calculate the FID between the test set and a random sample of 500 images used for training, to show that even if probably lower than compared to the ones calculated for the generated images, the FID will probably be very high.

In [ ]:
%mkdir /content/real

In [ ]:
dirpath= '/content/drive/MyDrive/Data/train/'
filenames = random.sample(os.listdir(dirpath), 500) 
for fname in filenames:
  source = dirpath+fname
  destination = '/content/real/' + fname
  shutil.copy(source, destination)

len(os.listdir('/content/real/'))

500

In [ ]:
!python -m pytorch_fid /content/drive/MyDrive/Data/test /content/real/ --device cuda:0

100% 10/10 [00:11<00:00,  1.18s/it]
100% 10/10 [00:10<00:00,  1.09s/it]
FID:  64.16747185713473


As expected, the FID between train and test images is pretty high.